# Import Packages

In [28]:
import pandas as pd

# Import Data

In [29]:
# Import the CSV to fighter and fight
fighter = pd.read_csv('../ufc_data_raw_main/raw_fighter_details.csv')
fight = pd.read_csv('../ufc_data_raw_main/raw_total_fight_data.csv', delimiter=';')

# Merge Datasets

In [30]:
# Add 'R' to column names for R_fighter and 'B' for Blue to differentiate in dataset.
R_fighter = fighter.add_prefix("R_")
B_fighter = fighter.add_prefix("B_")

# Merge datasets.
data = fight.merge(R_fighter, how='left', left_on='R_fighter', right_on='R_fighter_name')
data = data.merge(B_fighter, how='left', left_on='B_fighter', right_on='B_fighter_name')

# Clean Data

In [31]:
# Remove characters from Weight, Reach and Height

weight_cols = ['R_Weight','B_Weight']
reach_cols = ['R_Reach','B_Reach']
height_cols = ['R_Height','B_Height']

# Remove 'lbs.' from weight cols
for weight in weight_cols:
    data[weight] = data[weight].apply(lambda x: float(x.replace(' lbs.', '')) if type(x) != float else x)

# Remove " from reach cols
for reach in reach_cols:
    data[reach] = data[reach].apply(lambda x: float(x.replace('"', '')) if type(x) != float else x)
    
# Convert height from feet' inch" to cm using funciton to apply by lambda to column
def height_convert(height):
    ft_in = []
    height_cm = 0
    for i in height.split("'"):
        i = i.replace("'","")
        i = i.replace('"','')
        ft_in.append(i)
    if len(ft_in) == 2:
        height_cm += float(ft_in[0]) * 30.48
        height_cm += float(ft_in[1]) * 2.54
    else:
        height_cm += float(ft_in[0]) * 30.48
    return height_cm

for height in height_cols:
    data[height] = data[height].apply(lambda x: height_convert(x) if type(x) != float else x)

In [32]:
# Convert the DOB column to date time and create age column by subtracting DOB from today's date. 
data[['R_DOB','B_DOB']] = data[['R_DOB','B_DOB']].apply(pd.to_datetime)

data['R_current_age'] = pd.to_datetime(pd.Timestamp("today").strftime("%Y-%m-%d")) - data['R_DOB']
data['B_current_age'] = pd.to_datetime(pd.Timestamp("today").strftime("%Y-%m-%d")) - data['B_DOB']

# Year has 365 days, account for leap years, add 0.25. Day has 24 * 60 min, min has 60s. Multiply to get seconds in year.
data['R_current_age'] = data['R_current_age'].apply(lambda x: x.total_seconds() / (365.25*24*60*60))
data['B_current_age'] = data['B_current_age'].apply(lambda x: x.total_seconds() / (365.25*24*60*60))

In [33]:
# Remove percentae sign from the columns.

def remove_pct(pct):
    if type(pct) == str:
        return float(pct.replace('%',''))
    else:
        return float(pct)

pct_col = ['R_Str_Acc','B_Str_Acc','R_Str_Def','B_Str_Def',
           'R_TD_Acc','B_TD_Acc','R_TD_Def','B_TD_Def']    

for col in pct_col:
    data[col] = data[col].apply(lambda x: remove_pct(x))

In [34]:
# R_CTRL and B_CTRL and last_round_time to seconds.
# Note, we may be able to delete the else clause.

def get_sec(time):
    # Transform time mm:ss to seconds
    if time == '--':
        return None
    if len(time.split(':')) == 2:
        m, s = time.split(':')
        return int(m) * 60 + float(s)
    else:
        return None

time_col = ['R_CTRL','B_CTRL','last_round_time']

for col in time_col:
    data[col] = data[col].apply(lambda x: get_sec(x))

In [35]:
# Generate pct for all cols with 'X of Y' to generate percentages.
def generate_pct(string):
    if 'of' in string:
        num, den = string.split("of")
        if float(den) != 0:
            pct = (float(num)/float(den))*100
            return pct
        else:
            return None
    else:
        return None

pct_conv_cols = ['R_SIG_STR.','B_SIG_STR.','R_TOTAL_STR.','B_TOTAL_STR.',
               'R_TD','B_TD','R_HEAD','B_HEAD','R_BODY','B_BODY','R_LEG',
               'B_LEG','R_DISTANCE','B_DISTANCE','R_CLINCH','B_CLINCH',
               'R_GROUND','B_GROUND']    

for col in pct_conv_cols:
    data[f"{col}_pct"] = data[col].apply(lambda x: generate_pct(x))
    data.drop(columns=col, inplace=True)

In [36]:
# Code R winner (instead of a named winner) True if Red Wins, False if Blue Wins
data['R_Win'] = data['R_fighter'] == data['Winner']

In [37]:
# Drop Uneccessary Columns

data.drop(columns=['R_DOB','B_DOB','R_SIG_STR_pct',
                   'B_SIG_STR_pct','Referee', 'location',
                   'date','Winner','R_fighter_name',
                   'B_fighter_name'], inplace=True)

# Output

In [38]:
# Check columns and example values.
data_cols = list(data.columns)
for i in data_cols:
    print(f"{i}: {data[i][1]}")

R_fighter: Trevin Giles
B_fighter: Roman Dolidze
R_KD: 0
B_KD: 0
R_TD_pct: 50.0
B_TD_pct: 33.33333333333333
R_SUB_ATT: 1
B_SUB_ATT: 2
R_REV: 0
B_REV: 1
R_CTRL: 75.0
B_CTRL: 255.0
win_by: Decision - Unanimous
last_round: 3
last_round_time: 300.0
Format: 3 Rnd (5-5-5)
Fight_type: Middleweight Bout
R_Height: 182.88
R_Weight: 185.0
R_Reach: 74.0
R_Stance: Orthodox
R_SLpM: 3.26
R_Str_Acc: 56.0
R_SApM: 1.88
R_Str_Def: 62.0
R_TD_Avg: 1.37
R_TD_Acc: 80.0
R_TD_Def: 79.0
R_Sub_Avg: 0.3
B_Height: 187.96
B_Weight: 205.0
B_Reach: 76.0
B_Stance: Orthodox
B_SLpM: 2.98
B_Str_Acc: 51.0
B_SApM: 1.75
B_Str_Def: 61.0
B_TD_Avg: 1.75
B_TD_Acc: 50.0
B_TD_Def: 33.0
B_Sub_Avg: 2.2
R_current_age: 31.589322381930184
B_current_age: 35.64955509924709
R_SIG_STR._pct: 47.368421052631575
B_SIG_STR._pct: 47.76119402985074
R_TOTAL_STR._pct: 58.9041095890411
B_TOTAL_STR._pct: 68.18181818181817
R_HEAD_pct: 43.13725490196079
B_HEAD_pct: 27.027027027027028
R_BODY_pct: 100.0
B_BODY_pct: 50.0
R_LEG_pct: 50.0
B_LEG_pct: 93.75

In [39]:
data.to_csv("../data/data.csv", index=False)